In [1]:
# Import all necessary packages and functions here
# Make sure before running this, your selected python environment actually has all required packages.
from bs4 import BeautifulSoup
import configparser
import requests
import json
import pickle
import pandas as pd
import datetime
from constants_funcs import get_session, int_or_float_or_str
with open('constants.json', 'r') as d:
    constants = json.load(d) # check out this file to see all your options, and feel free to add more.

In [2]:
# This establishes your login session-do NOT run more than necessary or you may be booted from LL.
# If your login session is messed up/expires, set force_new_session=True--but again, no more than necessary.
sess = get_session(force_new_session=False)
sess # putting just a variable name on the last line of a cell will print its value.

using existing login session


In [8]:
# First, open the smaller files not in the .zip. JSON files opened normally, csv's opened with Pandas.
with open('./pp_by_username.json', 'r') as fp:
    id_lookup_by_username = json.load(fp)
full_question_history = pd.read_csv('./FULL_QUESTION_HISTORY_v2.csv', header=0)
player_profiles = pd.read_csv('./player_profiles.csv', header=0)

In [9]:
full_question_history.head() # head() takes one optional param, number of rows to show (default 5).

,index,season,matchday,qnum,question,answer,category,A_correct,B_correct,C_correct,D_correct,E_correct,R_correct,Champ_correct,tot_correct,defense,forfeit,mcw,mcw_pct,date
0,30-1-1,30,1,1,Mesothelioma is a form of cancer which is virt...,ASBESTOS,SCIENCE,71,69,41,NaN,NaN,71,NaN,61,1.5,NaN,NaN,NaN,10-Jul-06
1,30-1-2,30,1,2,Playwright Jonathan Larson died unexpectedly i...,RENT,THEATRE,88,86,56,NaN,NaN,71,NaN,75,1.5,NaN,NaN,NaN,10-Jul-06
2,30-1-3,30,1,3,"Former caterer, chef, author, entrepreneur, co...",BAREFOOT CONTESSA,FOOD/DRINK,50,47,28,NaN,NaN,29,NaN,40,1.7,NaN,NaN,NaN,10-Jul-06
3,30-1-4,30,1,4,"According the U.S. census of 1900, what were t...","NEW YORK, CHICAGO, PHILADELPHIA",AMER HIST,54,39,28,NaN,NaN,29,NaN,38,1.8,NaN,NaN,NaN,10-Jul-06
4,30-1-5,30,1,5,Identify the American novelist and author of t...,SUE GRAFTON,LITERATURE,67,58,44,NaN,NaN,43,NaN,54,1.7,NaN,NaN,NaN,10-Jul-06


In [7]:
player_profiles.head(3) # i wouldn't recomment doing more than 50, it can get messy and take too long to output.

,index,id,deceased,current_status,username,branch_id,branch_name,college,gender,location,referrer,league
0,15241,15241,False,deactivated,ShapiroJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11123,11123,False,deactivated,O'BrienM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39204,39204,False,std,AVeryMerryUnbirthday,25924.0,ThompsonAM,"University of California, Santa Cruz",Female,"Berlin, Germany",9113.0,Skyline


In [10]:
# Unzip the archive.zip file, then set the path variable below. If it's in the same folder, set path = './'.
path = './FULL_DATA' # mine happens to be in the FULL_DATA folder in my repo-change as needed.
leagues_by_season = pd.read_csv(f'{path}/leagues.csv') # for more info on f-formatted strings, see: https://realpython.com/python-f-strings/#f-strings-a-new-and-improved-way-to-format-strings-in-python
rundles_by_season = pd.read_csv(f'{path}/rundles.csv')
matches_history = pd.read_csv(f'{path}/matches_history.csv')
match_participation = pd.read_csv(f'{path}/match_participation.csv')
player_answer_history = pd.read_csv(f'{path}/player_answer_history.csv')
# Note that this may take a hot minute, considering the size of these (especially player_answer_history at 3.6 GB!)
# DtypeWarnings are OK for now, may be resolved in future versions.

/usr/local/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/usr/local/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
leagues_by_season.head() # pretty self-explanatory, only thing:
# when league = 'League', this rundle wasn't assigned a specific region: see ~LL30-41 for examples.

,season,league
0,30,League
1,31,League
2,32,League
3,33,League
4,34,League


In [12]:
rundles_by_season.head() # just tracks the existence of rundles by season-not yet sure if this is helpful...
# Note: when div = 0, that rundle was not split in divisions.
# For example, LL94 rundle A Central would have div = 0, but the E Central divisions would be 1, 2, and 3 (3 rows).

,season,rundle,league,div
0,30,A,League,0
1,30,B,League,0
2,30,C,League,0
3,30,R,League,0
4,31,A,League,0


In [13]:
matches_history.head() # this ties match IDs to specific match days and rundles.
# Be careful about using these match IDs, I do not believe they are fully unique.

,match_id,rundle,league,div,season,md
0,25904701,A,League,0,30,1
1,4305401,A,League,0,30,1
2,2302401,A,League,0,30,1
3,12625701,A,League,0,30,1
4,6618301,A,League,0,30,1


In [14]:
match_participation.head() # this ties players to matches they played, and their result for each.
# Note this is separate from questions answered, because questions answered can't track forfeits.
# Again, be wary of using match IDs alone for uniqueness purposes.

,match_id,season,md,rundle,league,div,player,result,player_id
0,25904701.0,30,1,A,League,0,white,L,19962
1,25904701.0,30,1,A,League,0,dages,W,28118
2,4305401.0,30,1,A,League,0,crainc,W,19713
3,4305401.0,30,1,A,League,0,doro,L,27800
4,2302401.0,30,1,A,League,0,brooke,W,20858


In [15]:
player_answer_history.head() # this shows each player-answer combo, whether they got it and the points they were given.
# NOTE: def_given is the points a player's OPPONENT assigned to that question, ie how many points THIS player would earn for the q.

,match_id,rundle,league,div,season,md,qnum,player,correct,def_given,player_id
0,25904701.0,A,League,0,30,1,1,white,True,2.0,19962
1,25904701.0,A,League,0,30,1,2,white,False,3.0,19962
2,25904701.0,A,League,0,30,1,3,white,True,1.0,19962
3,25904701.0,A,League,0,30,1,4,white,False,1.0,19962
4,25904701.0,A,League,0,30,1,5,white,False,2.0,19962


In [22]:
# filtering on one condition: <dataframe>.loc[(<dataframe>['column'] <operator> <value>), ?<columns>[] ]
player_answer_history.drop_duplicates().loc[(player_answer_history['player_id'] == 23168), ['qnum', 'correct', 'def_given']]

,qnum,correct,def_given
9026526,1,True,0.0
9026528,2,True,1.0
9026530,3,False,2.0
9026532,4,False,2.0
9026534,5,True,3.0
...,...,...,...
69467283,2,False,3.0
69467285,3,False,0.0
69467287,4,True,1.0
69467289,5,False,2.0


In [50]:
# filter table on multiple conditions
cond1 = player_answer_history['player_id'] == 23168
cond2 = player_answer_history['season'] >= 80
cond3 = player_answer_history['season'] == 74
player_answer_history.loc[(cond1 & (cond2 | cond3))]

,match_id,rundle,league,div,season,md,qnum,player,correct,def_given,player_id
12551286,1571689.0,E,Central,1,74,5,1,smitht04,True,3.0,23168
12551288,1571689.0,E,Central,1,74,5,2,smitht04,True,0.0,23168
12551290,1571689.0,E,Central,1,74,5,3,smitht04,False,1.0,23168
12551292,1571689.0,E,Central,1,74,5,4,smitht04,True,2.0,23168
12551294,1571689.0,E,Central,1,74,5,5,smitht04,False,2.0,23168
...,...,...,...,...,...,...,...,...,...,...,...
69467283,6379293.0,E,Central,2,94,25,2,23168,False,3.0,23168
69467285,6379293.0,E,Central,2,94,25,3,23168,False,0.0,23168
69467287,6379293.0,E,Central,2,94,25,4,23168,True,1.0,23168
69467289,6379293.0,E,Central,2,94,25,5,23168,False,2.0,23168


In [52]:
# join two dataframes: <df1>.join(<df2>, on?=<column>, how?=<join type>, lsuffix?=<suffix>, rsuffix=<suffix>)
matches_history.join(match_participation, on='match_id', how='inner', lsuffix='_mh', rsuffix='_mp')

,match_id,match_id_mh,rundle_mh,league_mh,div_mh,season_mh,md_mh,match_id_mp,season_mp,md_mp,rundle_mp,league_mp,div_mp,player,result,player_id
1,4305401,4305401,A,League,0,30,1,2713477.0,81,10,E,Magnolia,2,zimmermanr,T,7693
3863043,4305401,4305401,D,Hinterland,1,88,11,2713477.0,81,10,E,Magnolia,2,zimmermanr,T,7693
2,2302401,2302401,A,League,0,30,1,1618874.0,74,15,E,Seaboard,2,godwinj,W,2644
1629438,2302401,2302401,R,Horizon,2,78,10,1618874.0,74,15,E,Seaboard,2,godwinj,W,2644
4,6618301,6618301,A,League,0,30,1,3848954.0,86,19,R,Windward,0,47653,F,47653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5790047,6382286,6382286,E,Zephyr,1,94,25,3662285.0,86,1,D,Piedmont,2,44497,L,44497
5790048,6382287,6382287,E,Zephyr,1,94,25,3662285.0,86,1,D,Piedmont,2,42352,W,42352
5790049,6382288,6382288,E,Zephyr,1,94,25,3662286.0,86,1,D,Piedmont,2,18209,W,18209
5790050,6382289,6382289,E,Zephyr,1,94,25,3662286.0,86,1,D,Piedmont,2,39037,L,39037
